# Temperature plot Innsbruck

In [26]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import statsmodels.api as sm

### - Open the precipitation and temperature datasets

In [27]:
T_df = pd.read_table("/home/ciccuz/temperature_plot/HISTALP_AT_INN_T01_1760_2020.csv", sep=";", skiprows=13, index_col=0)
p_df = pd.read_table("/home/ciccuz/temperature_plot/HISTALP_AT_INN_R01_1760_2020.csv", sep=";", skiprows=13, index_col=0)

### - Some post-processing steps

In [28]:
#keep just the monthly values and not the seasonal ones
T_df1 = T_df.iloc[:,0:12]

#find all the data were no missing values are present
T_monthly = pd.DataFrame(index=range(1,2907), columns=['date', 'T_mean'])
T_mean = []
i=0
for year in T_df1.index:
    
    #array of values for the current year:
    x = T_df1.loc[year].loc[T_df1.loc[year].astype(np.float) != 999999]
    
    if x.size == 0:
        continue
    else:
        T_mean = pd.concat([pd.Series(T_mean), pd.Series(x.values)], axis=0).reset_index(drop=True)
        
        #cycle to pick the date: year and month
        for month in range(1,len(x)+1):
            T_monthly['date'][i+month] = pd.to_datetime(f'{year}{month}', format='%Y%m')

        i=i+12
        
T_mean.index += 1 

#multiply *0.1 to obtain actual degrees C
T_monthly['T_mean'] = T_mean*0.1

### - Choose the year range

In [52]:
#choose the 100 years on which calculate means, do the regression and plot: NOT LARGER THAN 1918!
first_year = 1915
index_first_year = T_monthly.loc[(T_monthly['date'].dt.year == first_year)].index[0]
index_last_year = T_monthly.loc[(T_monthly['date'].dt.year == first_year+100)].index[-1]

T_monthly_subset = T_monthly.loc[index_first_year : index_last_year].reset_index(drop=True)

### - Calculate the annual mean values and do the linear regression

In [53]:
#Annual mean (mean values every 12 months)
T_annual_avg = pd.DataFrame(columns=['year', 'T_mean'])
T_annual_avg['year'] = pd.to_datetime(np.unique(T_monthly_subset['date'].dt.year), format='%Y')

for year,i in zip(np.unique(T_monthly_subset['date'].dt.year), range(0,101,1)):
    T_annual_avg['T_mean'][i] = np.mean(T_monthly_subset['T_mean'].loc[(T_monthly_subset['date'].dt.year == year)])


#Linear regression of monthly means values:
var_x = T_monthly_subset['date'].apply(lambda date: date.toordinal())
var_y = T_monthly_subset['T_mean']
data = np.transpose([var_x, var_y])
df_linreg = pd.DataFrame(data, columns=['date', 'T_mean'])

results = sm.OLS(df_linreg.T_mean, sm.add_constant(df_linreg.date)).fit()
par = results.params

# Plot

In [54]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import SingleIntervalTicker, LinearAxis, tickers

In [55]:
output_notebook()

Loading BokehJS ...

In [56]:
title = f'Temperature in Innsbruck {first_year}-{first_year+100}'

# create a new plot with a title and axis labels
p = figure(title=title, x_axis_type="datetime", plot_width=950, plot_height=400, 
           x_range=[T_annual_avg['year'][0], T_annual_avg['year'].iloc[-1]])
p.title.text_font_size='20pt'

#plot the monthly means time-series
Tmm = p.line(T_monthly_subset['date'], T_monthly_subset['T_mean'], legend="T monthly avg ", line_width=1.5, alpha=0.5, 
              muted_alpha=0.2)

#plot the average values over the 100 years
T100m = p.line(T_monthly_subset['date'], np.mean(T_annual_avg['T_mean']), legend="T 100-years avg ", line_width=2, 
       color = 'green', alpha=0.8,  muted_alpha=0.2, muted_color='green')

#linear regression line:
Ttrend = p.line(T_monthly_subset['date'], par.const + par.date * df_linreg.date, legend="T trend", line_width=2, 
                color = 'red',  muted_alpha=0.2, muted_color='red')

#plot the yearly means time-series
Tym = p.line(T_annual_avg['year'], T_annual_avg['T_mean'], legend="T annual avg ", line_width=1.5, color = 'black',
              muted_alpha=0.2, muted_color='black')

#set the x and y grid lines
p.xgrid.grid_line_alpha=0.8
p.ygrid.grid_line_alpha=0.8

#set y label
p.yaxis.axis_label='°C'

#set the major and minor x ticks frequency
p.xaxis[0].ticker.desired_num_ticks = 20
p.axis.major_tick_out = 10

#customize the legend
p.legend.orientation = "horizontal"
p.legend.location = "top_center"
p.legend.border_line_width = 1.5
p.legend.border_line_color = "black"
p.legend.border_line_alpha = 0.5

#interactive legend: by clicking on the legend element it is muted
p.legend.click_policy="mute"

show(p)